In [1]:
#############  u===2   MLP_encode_100 and MLP_encode_1000  :654    ##################
import torch
#import Trainer
from network import NFM
import torch.utils.data as Data
from Utils.criteo_loader import getTestData, getTrainData
from torch.autograd import Variable
from torch.utils.data import DataLoader
from sklearn.metrics import roc_auc_score
from new_nfm_network import NFM
import os
import time
import argparse
import numpy as np
import pandas as pd 
import sys

import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data
import torch.backends.cudnn as cudnn
#3from tensorboardX import SummaryWriter
import sys 
#import network
import config
import evaluate
import data_utils
import Trainer
import torchmetrics

from innvestigator import InnvestigateModel
from utils import Flatten
import torch

import torch.nn as nn
import torch.optim as optim


from torch.utils.data import DataLoader
from torchvision import datasets, transforms

from mnist_test import Net, train, test

input_num=244
# Network parameters
class Params(object):
    batch_size = 64
    test_batch_size = 20
    epochs = 5
    lr = 0.01
    momentum = 0.5
    no_cuda = True
    seed = 1
    log_interval = 10
    
    def __init__(self):
        pass

args = Params()
torch.manual_seed(args.seed)
#device = torch.device("cpu")
device=torch.device('cuda')
kwargs = {}
nfm_config = \
{
    'n_class':9,
    
    'num_epoch': 500,#训练epoch次数
    'batch_size': 16,#batch_size
    'lr': 1e-3,
    'l2_regularization': 1e-4,
    'device_id': 0,
    'use_cuda': False,
    'epoch':1000,
    
   
    'gene_name':'dataset/qiuguan/origin_800/gene_name.csv',
    'label_name':'dataset/qiuguan/origin_800/gene_label.csv'
    
}

#model definition
import torch.nn as nn
    
    

#model1 = MLP().cuda()
#print(model1)

class MLP_P(nn.Module):
    def __init__(self):
        super().__init__()
        self.bn0=nn.BatchNorm1d(input_num)
        self.fc1 = nn.Linear(input_num, 1000)
        self.bn1= nn.BatchNorm1d(1000)
        self.fc2 = nn.Linear(1000, 100)
        self.bn2=nn.BatchNorm1d(100)
        self.fc3=nn.Linear(100,9)
        self.bn3=nn.BatchNorm1d(9)
        
        self.drop=nn.Dropout(0.3)    


class MLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.bn0=nn.BatchNorm1d(input_num)
        self.fc1 = nn.Linear(input_num, 1000)
        self.bn1= nn.BatchNorm1d(1000)
        self.fc2 = nn.Linear(1000, 100)
        self.bn2=nn.BatchNorm1d(100)
        self.fc3=nn.Linear(100,9)
        self.bn3=nn.BatchNorm1d(9)
        
        self.drop=nn.Dropout(0.3) 
        
        #self.model1=MLP1().cuda() 
        
        
        
    def forward(self,x):
        
        y1=self.bn0(x)
        y1 = F.relu(self.drop(self.bn1(self.fc1(y1))))
        y1= F.relu(self.drop(self.bn2(self.fc2(y1))))
        return F.softmax(self.bn3(self.fc3(y1)), dim=1)
        
           
        
        
        
   
    
    
    
model = MLP().cuda()

for i in model.state_dict():
    print(i)
print(model)
mlp_paras=list(model.named_parameters())
#print(mlp_paras)
inn_model = InnvestigateModel(model, lrp_exponent=2,
                              method="e-rule",
                              beta=.5)

import os
import time
import argparse
import numpy as np
import pandas as pd 

import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data
import torch.backends.cudnn as cudnn
#3from tensorboardX import SummaryWriter
import torch.nn.functional as F  # 激励函数的库
#import network
import config
import evaluate
import data_utils
import Trainer

def one_hot_smoothing(labels, classes, label_smoothing=0.2):
    #n = len(labels)
    n=labels.shape[0]
    eoff = label_smoothing / classes
    output = np.ones((n, classes), dtype=np.float32) * eoff
    for row, label in enumerate(labels):
        output[row, label] = 1 - label_smoothing + eoff
        #print("row:",row,"label:",label)
    return output

def one_hot(labels, classes):
    n = len(labels)
    #eoff = label_smoothing / classes
    output = np.zeros((n, classes), dtype=np.float32)
    for row, label in enumerate(labels):
        output[row, label] = 1
        #print("row:",row,"label:",label)
    return output


import pandas as pd 

from sklearn.model_selection import train_test_split




def one_hot_smoothing(labels, classes, label_smoothing=0.2):
    #n = len(labels)
    n=labels.shape[0]
    eoff = label_smoothing / classes
    output = np.ones((n, classes), dtype=np.float32) * eoff
    for row, label in enumerate(labels):
        output[row, label] = 1 - label_smoothing + eoff
        #print("row:",row,"label:",label)
    return output

def one_hot(labels, classes):
    n = len(labels)
    #eoff = label_smoothing / classes
    output = np.zeros((n, classes), dtype=np.float32)
    for row, label in enumerate(labels):
        output[row, label] = 1
        #print("row:",row,"label:",label)
    return output

class KZDatasetTest(data.Dataset):
    """ Construct the FM pytorch dataset. """
    #def __init__(self, file,label_file, feature_map,n_class=16):
    def __init__(self, csv_path):
    
       
        self.data_info = self.get_data_info(csv_path)
        
        
            
        
        
        

    def __getitem__(self, index):
        # Dataset读取图片的函数
        data, label = self.data_info[index]
        #img = Image.open(img_pth).convert('RGB')
        
        return data, label

    def __len__(self):
        return len(self.data_info)
    
    
    
    
    def get_data_info(self,csv_path):
        #解析路径
        #转为一维list存储，每一位为【图片路径，图片类别】
        labels=[]
        data_info=[]
        df=pd.read_csv(csv_path,sep=',',header=None)
        #print("df:",df)
        df=df.iloc[2:,1:]
        
        rows,cols=df.shape
        #print(rows,cols)
        for i in df.iloc[:,-1]:
            #print(i)
            labels.append(int(float(i)))
        #print('labels:',labels)
        labels=np.array(labels)
        
        labels=one_hot_smoothing(labels,nfm_config['n_class'])
        #print(labels)
        for i in range(rows):
            data=df.iloc[i,:-1]
            data=data.astype(float)##############
            
            data=np.array(data)##
            
            label=labels[i]
            
            data=torch.from_numpy(data)#
            label=torch.from_numpy(label)#
           
            
            data_info.append((data,label))
        return data_info
    
    

    
import torch
import torch.nn as nn
from torch.utils.data.dataset import *
from PIL import Image
from torch.nn import functional as F
import random
from sklearn.model_selection import train_test_split
import ast
import torchvision


        

class KZDataset(Dataset):
    def __init__(self, csv_path, K,n_class,ki=0, typ='train', transform=None, rand=False):
        
        self.all_data_info = self.get_data_info(csv_path)
        
        if rand:
            random.seed(1)
            random.shuffle(self.all_data_info)
        leng = len(self.all_data_info)
        every_z_len = leng // K
        if typ == 'val':
            self.data_info = self.all_data_info[every_z_len * ki : every_z_len * (ki+1)]
        elif typ == 'train':
            self.data_info = self.all_data_info[: every_z_len * ki] + self.all_data_info[every_z_len * (ki+1) :]
            
        self.transform = transform
        
        

    def __getitem__(self, index):
        # Dataset读取图片的函数
        data, label = self.data_info[index]
        #img = Image.open(img_pth).convert('RGB')
        
        return data, label

    def __len__(self):
        return len(self.data_info)
       
    
    def get_data_info(self,csv_path):
        #解析路径
        #转为一维list存储，每一位为【图片路径，图片类别】
        labels=[]
        data_info=[]
        df=pd.read_csv(csv_path,sep=',',header=None)
        #print("df:",df)
        df=df.iloc[2:,1:]
        #print("df:",df)
        print(df.shape)
        #print("df:",df)
        #print(df.iloc[:,-1])
        #df=df.applymap(ast.literal_eval)
        rows,cols=df.shape
        #print(rows,cols)
        for i in df.iloc[:,-1]:
            #print(i)
            labels.append(int(float(i)))
        #print('labels:',i,labels)
        labels=np.array(labels)
        #print('labels:',labels)
        #labels=np.array(labels)
        labels=one_hot_smoothing(labels,nfm_config['n_class'])
        #print(labels)
        for i in range(rows):
            data=df.iloc[i,:-1]
            data=data.astype(float)##############
            #print("i,data:",i,data)
            #data=pd.DataFrame(data,dtype=float)###############
            data=np.array(data)##
            
            label=labels[i]
            #print(data.shape)
            #print(label.shape)
            #label=label.tolist()
            data=torch.from_numpy(data)#
            label=torch.from_numpy(label)#
           
            
            data_info.append((data,label))
        return data_info

    

        
from torch.autograd import Variable
from torch.utils.data import DataLoader
from sklearn.metrics import roc_auc_score
from new_nfm_network import NFM
import os
import time
import argparse
import numpy as np
import pandas as pd 
import sys

import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data
import torch.backends.cudnn as cudnn
#3from tensorboardX import SummaryWriter
import sys 
#import network
import config
import evaluate
import data_utils
import Trainer
import torchmetrics
#LRP
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
%matplotlib inline

from innvestigator import InnvestigateModel
from utils import Flatten
def standN(x):
    row,col=x.shape
    max_x=torch.max(x,1)
    min_x=torch.min(x,1)
    #print(max_x)
    #print(min_x)
    for i in range(row):
        x[i]=(x[i]-min_x.values[i])/max_x.values[i]
    return x

def standNorm(x):
    row,col=x.shape
    mean=torch.mean(x,1)
    std=torch.std(x,1)
    #print(mean)
    #print(std)
    for i in range(row):
        x[i]=(x[i]-mean[i])/std[i]
    return x

def augment_epoch(kii,train_loader,batch_size):
    BATCH_SIZE=batch_size
    total = 0
    correct=0
    total_loss=0
    #loss_score=torch.tensor([[]]).cuda()
    #
    #loss_op=0
    loss2_list=[]
    model.train()
    total_train_accuracy=0  
    #BL=nn.BatchNorm1d(input_num)
    #BL=BL.cuda()
    y=[[0]*input_num]*batch_size
    y=torch.tensor(y,dtype=torch.float)
    y=y.cuda()
    
    for batch_idx, (x, labels) in enumerate(train_loader):
            
        labels = Variable(labels)
        x = Variable(x)
            
        x_row,x_col=x.shape   
        x=torch.tensor(x,dtype=torch.float)
        #print(x.shape)
        labels=torch.tensor(labels,dtype=torch.float)
        x, labels = x.cuda(), labels.cuda()
        labels_int=labels=torch.max(labels,1)[1]#########
        #print(labels_int)
        #print('labels_int:',labels_int.shape)
        #print('labels:',labels) 
        #print('x:',x.shape)
        ll=labels.view(batch_size,-1)
        #ll.dtype=torch.int
        #print('ll:',ll)
        if kii>=0 : 
            
            inn_model.evaluate(in_tensor=x)
        
            model_prediction, only_max_score,org_shape = inn_model.innvestigatex()
            model_prediction.cuda()
            #print('only_max_score:',only_max_score)
            #only_max_score1=only_max_score.detach().clone()
            #print(labels.shape,labels)   
        
            #print("torch.argmax(labels[batch_idx]):",torch.argmax(labels[batch_idx]))
            #print(model_prediction.shape, model_prediction)
            #model_prediction.cuda()
            #print('input_relevance_values:')
            rel_for_class_list=torch.argmax(model_prediction,1)
            #print(rel_for_class_list)
            para_list=[]
            
            '''  
            rand=torch.linspace(0, 1, steps=input_num)
            rand=rand.detach().numpy().tolist()
            
            para_list=[round(i,4) for i in rand]
            '''
            
            #print(para_list)
            
            
            for i in range(batch_size):
            
                max_pre=torch.argmax(model_prediction[i])
                if max_pre!=labels[i]:
                    #print('only_max_score1:',i,only_max_score)
                    #only_max_score=only_max_score1
                    #input_relevance_values=inn_model.compute_relevance_score(only_max_score,labels[i],org_shape,para=0.1)
                    #rel_for_class=labels[i]
                    #para=random.gauss(0.6,0.4)#########0.6，0.4：98  96
                    
                    #para=random.gauss(0.6,0.7)############0.6  0.7  96  95
                    
                    #para=random.gauss(0.7,0.4)###############
                    #para=random.gauss(0.7,0.3)#############9110  96  98，0.2，0.1
                    #para=random.gauss(0.6,0.5)########96  98  0.1  0.1  9110
                    #para=random.gauss(0.6,0.2)#######0.1 0.1  96  96
                    para=random.gauss(0.6,0.4)############7510pkl   96  96  0.6  0.4  0.1  0.1
                    para=round(para,4)
                    para_list.append(para)
                    #print('device:',input_relevance_values.device)
                    #input_relevance_values[labels[i]]=input_relevance_values[labels[i]].cuda()
                    #input_relevance_values[labels[i]]=input_relevance_values[labels[i]].exp()
                    #print('input_relevance_values:',i, input_relevance_values)
                    #print('input_relevance_values[labels[i]]:',i,batch_idx,input_relevance_values[labels[i]])
                    #cc[i]=torch.mul(x[i],input_relevance_values[labels[i]])+torch.tensor(0.1,dtype=torch.float)##############注意力
                    #print('pre_target:',pre_target[i])
                    #print('cc',i,cc[i].shape)
                else:
                    #print('only_max_score2:',i,only_max_score)
                    #only_max_score=only_max_score1
                    #input_relevance_values=inn_model.compute_relevance_score(only_max_score,labels[i],org_shape,para=0)
                    #input_relevance_values[labels[i]]=input_relevance_values[labels[i]].exp()
                    #print('input_relevance_values:',i, input_relevance_values)
                    #print('input_relevance_values[i]:',i,batch_idx,input_relevance_values[labels[i]])
                    #cc[i]=torch.mul(x[i],input_relevance_values[labels[i]])##############注意力
                    #print('cc',i,cc[i].shape)
                    para=random.gauss(0.1,0.2)
                    para=round(para,4)
                    para_list.append(para)
            
            
            
            input_relevance_values,layers=inn_model.compute_relevance_scorex(only_max_score,rel_for_class_list,org_shape,para_list)
            input_len=len(input_relevance_values)
            
            ee=[[0]*input_num]*batch_size
            ee=torch.tensor(ee,dtype=torch.float)
        
            ee=ee.cuda()
            input_relevance_values[-1]=input_relevance_values[-1].exp()############
            
            
            #sum_input_relevance_values=torch.sum(input_relevance_values,dim=0)
            #input_relevance_values=F.softmax(input_relevance_values/sum_input_relevance_values,dim=1)
            input_relevance_values[-1]=F.softmax(input_relevance_values[-1],dim=1)################
        
            '''
            for i in range(batch_size):
                        max_pre=torch.argmax(model_prediction[i])######
                        if max_pre!=labels[i]: #and i==min_predict_mis:###
                            
                                
            
                                ee[i]=torch.mul(x[i],input_relevance_values[-1][i])##############注意力33333333333333333weight
                                #dd=torch.mul(bias,input_relevance_values[-1][i])############bias
                                #print('cc:',i,cc)
                                #ee[i]=torch.mul(drop,input_relevance_values[i])
                                ################
            '''                    
            ee=torch.mul(x,input_relevance_values[-1])
            
            
            x1=torch.subtract(x,ee)
            x2=torch.add(x,ee)
            #print(ll.shape)
            #print(x.shape)
            #x1=torch.cat((x1,ll),dim=1)
            #x2=torch.cat((x2,ll),dim=1)
            
            l1=ll
            l2=ll
            if batch_idx==0:
                
                y=torch.cat((x1,x2),dim=0)
                l=torch.cat((l1,l2),dim=0)
            y=torch.cat((y,x1),dim=0)
            y=torch.cat((y,x2),dim=0)
            
            l=torch.cat((l,l1),dim=0)
            l=torch.cat((l,l2),dim=0)
    
    
            
           
            
            
                    
            
            
            
            
              
            
            
                
                
                
                        
        
        
        
       
        
        
        
    '''  
        #print('x_new:',x)
        #print('new_x:',x)
        optimizer.zero_grad()
        y_predict=model(x)
        #y_predict=model(x,labels)
            
        
        
        
        loss1 = loss_func(y_predict, labels)
        #print('loss1:',loss1)
        #loss2=loss_func(y_predict1,labels)
        #loss2=u*(1/loss_op)
        #print('input_relevance_values:',input_relevance_values.shape)
        #print('loss_score:',loss_score.shape)
        #cc=1.0/torch.abs(torch.sum(cc))
        
        
        
        
        #loss2=u*cc
        #print('input_relevance_values:',input_relevance_values)
        
        #print('loss1:',loss1)
        #print('loss2:',loss2)
        #loss=loss1+loss2
        loss=loss1
        #loss = loss_func(y_predict, labels)
        loss.backward()
        optimizer.step()
        #loss2_list.append(u*loss2)   
        loss = loss.item()
           

        total_loss += loss
            
            
            
        batch_train_acc=torchmetrics.functional.accuracy(y_predict,labels_int)
        total_train_accuracy+=batch_train_acc
    #plotLoss(loss2_list,batch_idx+1)   #################################     
    total_train_accuracy/=(batch_idx+1)
    print('total_train_accuracy:',total_train_accuracy)
    print("Training Epoch: %d, total loss: %f" % (epoch_id, total_loss))
    return total_loss,total_train_accuracy
    
    ''' 
    return y,l

def val_epoch(test_loader,batch_size,optimizer): 
    batch_size_num=0
    total_test_acc=0
    model.eval()
    for i , (inputs , targets) in enumerate(test_loader):   
        print("val")
            
            
        inputs = Variable(inputs)   
        targets = Variable(targets)     
           
        inputs=torch.tensor(inputs ,dtype=torch.float)   
        targets=torch.tensor(targets ,dtype=torch.float)   
        inputs , targets = inputs.cuda(),  targets.cuda()
        targets=torch.max(targets,1)[1]
        
        
        
        
        
        optimizer.zero_grad()
        #yhat = model1(inputs,targets)
        yhat=model(inputs)
            
            
            
        #targets=torch.max(targets,1)[1]
            
            
            
        batch_test_acc=torchmetrics.functional.accuracy(yhat,targets)
            
        total_test_acc+=batch_test_acc
            
        batch_size_num=i
    total_test_acc/=(batch_size_num+1)
        ###print('total_test_accuracy:',total_test_acc/(batch_size+1))
    print('total_test_accuracy:',total_test_acc)
        
                    
                    
            
            
    
        
   
    
    return total_test_acc


import numpy as np
import math
import matplotlib.pyplot as plt

def plotLoss(loss,epoch):
    plt.rcParams['font.sans-serif']=['SimHei']
    plt.rcParams['axes.unicode_minus'] = False
    plt.figure(figsize=(8, 5))
    x=[i for i in range(epoch)]
    #acc_train=acc_train.cpu()
    #acc_test=acc_test.cpu()
    plt.plot(x, loss, 'r-', mec='k', label='Logistic Loss', lw=2)
    #plt.plot(x,acc_train,'b-',mec='k',label='accuracy Train',lw=2)
    #plt.plot(x,acc_test,'g-',mec='k',label='accuracy Test',lw=2)
    #plt.plot(x, y_01, 'g-', mec='k', label='0/1 Loss', lw=2)
    #plt.plot(x, y_hinge, 'b-',mec='k', label='Hinge Loss', lw=2)
    #plt.plot(x, boost, 'm--',mec='k', label='Adaboost Loss',lw=2)
    plt.grid(True, ls='--')
    plt.legend(loc='upper right')
    plt.title('损失函数')
    plt.show()

bn0.weight
bn0.bias
bn0.running_mean
bn0.running_var
bn0.num_batches_tracked
fc1.weight
fc1.bias
bn1.weight
bn1.bias
bn1.running_mean
bn1.running_var
bn1.num_batches_tracked
fc2.weight
fc2.bias
bn2.weight
bn2.bias
bn2.running_mean
bn2.running_var
bn2.num_batches_tracked
fc3.weight
fc3.bias
bn3.weight
bn3.bias
bn3.running_mean
bn3.running_var
bn3.num_batches_tracked
MLP(
  (bn0): BatchNorm1d(244, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc1): Linear(in_features=244, out_features=1000, bias=True)
  (bn1): BatchNorm1d(1000, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc2): Linear(in_features=1000, out_features=100, bias=True)
  (bn2): BatchNorm1d(100, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc3): Linear(in_features=100, out_features=9, bias=True)
  (bn3): BatchNorm1d(9, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (drop): Dropout(p=0.3, inplace=False)
)
relevance


In [2]:
#_,model=MLPA().cuda
import torch

#功能：加载保存到path中的各层参数到神经网络

path='dataset/qiuguan/origin_800/LRP/non_encode/80/without_attention/MLP1010.pkl'

#nfm=NFM(nfm_config)
mlp=MLP()
#print(nfm)
#net = nn.DataParallel(net)
#net = net.to(device)
mlp.load_state_dict(torch.load(path),strict=False)
mlp.cuda()

print(mlp)


#print(model.state_dict())

mlp_params = list(mlp.named_parameters())
#print(mlp_params)
model=mlp




inn_model = InnvestigateModel(model, lrp_exponent=2,
                              method="e-rule",
                              beta=.5)


import itertools

K=10
test_metrics=[]
train_loss_total_list=[]

testset= KZDatasetTest(csv_path='dataset/qiuguan/origin_800/LRP/80/selected_train_val_info.csv')
   
test_loader = data.DataLoader(
         dataset=testset,
         #transform=torchvision.transforms.ToTensor(),
         drop_last=True,
         batch_size=nfm_config['batch_size']
        
     )


#model_path='dataset/qiuguan/origin_800/LRP/non_encode/200/attention0.01/'
#BATCH_SIZE=batch_size
#optimizer = torch.optim.Adam(model.parameters(), lr=nfm_config['lr'], weight_decay=nfm_config['l2_regularization'])
#total = 0
    
    
#loss_func=torch.nn.CrossEntropyLoss()
    
num=0
   
model_named_parameters=[j for j in model.named_parameters()]
#print('model_parameters:',model_named_parameters)
#print('model.state.dict:',model.state_dict())
epoches=1
for epoch_id in range(epoches):
          
        
        
        y,l=augment_epoch(0,test_loader,nfm_config['batch_size'])
        #train_loss_total_list.append(train_loss_total)#
        
        '''  
        if epoch_id %10==0:
            num=num+1
            path=os.path.join(model_path,'MLP'+str(num)+str(K)+'.pkl')
            torch.save(model.state_dict(),path)
            
         '''  
y=y.detach().cpu().numpy()
y=pd.DataFrame(y)

l=l.detach().cpu().numpy()
l=pd.DataFrame(l)
#print(y)
#print(l)


aug=pd.concat((y,l),axis=1)
aug.to_csv('dataset/qiuguan/origin_800/LRP/80/aug_train_val_info.csv')
print(aug)

MLP(
  (bn0): BatchNorm1d(244, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc1): Linear(in_features=244, out_features=1000, bias=True)
  (bn1): BatchNorm1d(1000, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc2): Linear(in_features=1000, out_features=100, bias=True)
  (bn2): BatchNorm1d(100, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc3): Linear(in_features=100, out_features=9, bias=True)
  (bn3): BatchNorm1d(9, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (drop): Dropout(p=0.3, inplace=False)
)
relevance


/home/zhengfang/.local/lib/python3.6/site-packages/ipykernel_launcher.py:440: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
/home/zhengfang/.local/lib/python3.6/site-packages/ipykernel_launcher.py:442: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


           0         1          2         3         4         5         6    \
0     4.712716  4.719800   7.783406  6.887787  5.105534  6.448301  8.770288   
1     4.872344  4.655252   5.551160  6.862397  5.264141  7.233065  8.792705   
2     5.121512  4.722118   5.009521  7.260148  4.498782  9.396037  8.355294   
3     5.203842  5.147714   9.359057  8.069266  5.415675  7.978174  8.908166   
4     5.093595  4.635318   8.815096  6.958769  5.042191  6.857650  8.817730   
...        ...       ...        ...       ...       ...       ...       ...   
1115  5.549198  4.963156  11.240979  7.623978  5.662553  7.785930  8.764916   
1116  5.767074  5.232141  10.508600  6.777960  5.998055  7.694820  8.074890   
1117  5.450021  5.427273   9.696732  8.001414  5.380512  6.583365  7.515232   
1118  5.315147  5.105342  10.904235  6.793604  5.469623  6.246966  9.170125   
1119  6.099760  5.341326  14.266402  7.168535  5.111259  7.491924  8.306478   

            7         8         9    ...       235 

In [3]:
x=pd.read_csv('dataset/qiuguan/origin_800/LRP/80/selected_train_val_info.csv',sep=',')
#print(x)

#x=x.iloc[:,1:]

#print(x)

column=x.iloc[0,1:]#列名
#print(column)

x_new=x.iloc[1:,1:]
x_new.columns=column
#print(x_new)
aug.columns=column
#print(aug)

all=pd.concat((aug,x_new),axis=0)
print(all)

all.to_csv('dataset/qiuguan/origin_800/LRP/80/all_train_val_info.csv')

0            TYR        PPM1E                REN               ABCC3  \
0        4.71272       4.7198            7.78341             6.88779   
1        4.87234      4.65525            5.55116              6.8624   
2        5.12151      4.72212            5.00952             7.26015   
3        5.20384      5.14771            9.35906             8.06927   
4         5.0936      4.63532             8.8151             6.95877   
..           ...          ...                ...                 ...   
541  5.743529064  5.210875345        10.46588395  6.7502614670000005   
542  5.427847751  5.405209976  9.657287377000001         7.968845876   
543  5.293333995  5.084620506        10.85997562   6.765459452999999   
544   6.07443638  5.319655345        14.20590899         7.139406749   
545   5.19511412  4.927471871         6.83466967  7.9497790739999985   

0           LRP5           IGLV1-40       DNAJB1              ABCD3  \
0        5.10553             6.4483      8.77029            9.62